In [22]:
import pandas as pd
import math
import subprocess
import os
import warnings
warnings.filterwarnings("ignore")
from pydub import AudioSegment
from tqdm.notebook import tqdm

In [23]:


def get_or_convert_audio(audio_dir_path, audio_file_name):
    # Try .wav first
    wav_path = os.path.join(audio_dir_path, f"{audio_file_name}.wav")
    if os.path.exists(wav_path):
        return wav_path

    # Try other formats
    for ext in ['.m4a', '.webm']:
        alt_path = os.path.join(audio_dir_path, f"{audio_file_name}{ext}")
        if os.path.exists(alt_path):
            # Convert to wav using ffmpeg
            # Output to the same directory with .wav extension
            cmd = [
                'ffmpeg', '-y', '-i', alt_path, wav_path
            ]
            try:
                subprocess.run(cmd, capture_output=True, text=True, check=True)
            except subprocess.CalledProcessError as e:
                raise FileNotFoundError(f"ffmpeg failed for {alt_path}: {e.stderr}")
            return wav_path

    # If none exist, raise an error
    raise FileNotFoundError(f"No audio file found for {audio_file_name} in {audio_dir_path}")

In [30]:
def preprocess_park_celeb(audio_dir_path):
    patient_id, audio_file_name = audio_dir_path.split("/")[-2:]
    speaker_info = os.path.join(audio_dir_path, "speakers_info.csv")
    try:
        speaker_dataframe = pd.read_csv(speaker_info)
    except FileNotFoundError:
        return (patient_id, audio_file_name)
    target_dataframe = speaker_dataframe[speaker_dataframe['status'] == 'target']
    target_speaker = target_dataframe["speakers"].iloc[0]
    if target_dataframe["before_after_diagnosis"].iloc[0] == "before":
        yfd = target_dataframe["years_from_diagnosis"].iloc[0]
        if not math.isnan(yfd):
            yfd = int(yfd) * -1
        else:
            yfd = "None"
    else:
        yfd = target_dataframe["years_from_diagnosis"].iloc[0]
        if not math.isnan(yfd):
            yfd = int(yfd)
        else:
            yfd = "None"
    try:
        diarization = pd.read_csv(os.path.join(audio_dir_path, f"{audio_file_name}.csv"))
    except FileNotFoundError:
        return (patient_id, audio_file_name)
    speaker_diarization = diarization[diarization['speaker'] == target_speaker]
    speaker_diarization["length"] = speaker_diarization["end"] - speaker_diarization["start"]
    eight_sec_mask = speaker_diarization["length"] >= 8
    speaker_diarization = speaker_diarization[eight_sec_mask]
    try:
        input_audio_file = get_or_convert_audio(audio_dir_path, audio_file_name)
    except FileNotFoundError:
        return (patient_id, audio_file_name)
    temp_output = os.path.join(audio_dir_path, f"{patient_id}_{audio_file_name}_{yfd}_temp.wav")
    audio = AudioSegment.from_file(input_audio_file)
    combined = AudioSegment.empty()
    slices = []
    for start, end in zip(speaker_diarization["start"], speaker_diarization["end"]):
        slices.append((start*1000, end*1000))
    for start,end in slices:
        combined += audio[start:end]
    output_audio_file = os.path.join("Preprocessed Park Celeb Files", f"{patient_id}_{audio_file_name}_{yfd}.wav")
    combined.export(temp_output, format="wav")
    subprocess.run(["sox", temp_output, "-r", "16000", output_audio_file, "gain", "-n"])
    subprocess.run(["rm", temp_output])




In [31]:
park_celeb_dir = "/Users/thomas.cong/Downloads/ParkCeleb"
PD_patients = os.path.join(park_celeb_dir, "PD")
CN_patients = os.path.join(park_celeb_dir, "CN")
failed = []
for patient in tqdm(os.listdir(PD_patients)):
    if "pd" in patient:
        patient_folder = os.path.join(PD_patients, patient)
        for file in tqdm(os.listdir(patient_folder)):
            file_path = os.path.join(patient_folder, file)
            if os.path.isdir(file_path):
                result = preprocess_park_celeb(file_path)
                if result:
                    failed.append(result)
for patient in tqdm(os.listdir(CN_patients)):
    if "cn" in patient:
        patient_folder = os.path.join(CN_patients, patient)
        for file in tqdm(os.listdir(patient_folder)):
            file_path = os.path.join(patient_folder, file)
            if os.path.isdir(file_path):
                result = preprocess_park_celeb(file_path)
                if result:
                    failed.append(result)
print(failed)

  0%|          | 0/41 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

sox WARN rate: rate clipped 22 samples; decrease volume?
sox WARN dither: dither clipped 20 samples; decrease volume?
sox WARN rate: rate clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 2 samples; decrease volume?
sox WARN rate: rate clipped 40 samples; decrease volume?
sox WARN dither: dither clipped 32 samples; decrease volume?
sox WARN rate: rate clipped 40 samples; decrease volume?
sox WARN dither: dither clipped 38 samples; decrease volume?
sox WARN rate: rate clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 2 samples; decrease volume?


  0%|          | 0/23 [00:00<?, ?it/s]

sox WARN rate: rate clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 2 samples; decrease volume?
sox WARN rate: rate clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 2 samples; decrease volume?
sox WARN rate: rate clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 1 samples; decrease volume?
sox WARN rate: rate clipped 1 samples; decrease volume?
sox WARN dither: dither clipped 1 samples; decrease volume?
sox WARN rate: rate clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 2 samples; decrease volume?
sox WARN rate: rate clipped 6 samples; decrease volume?
sox WARN dither: dither clipped 6 samples; decrease volume?


...

sox WARN rate: rate clipped 196 samples; decrease volume?
sox WARN dither: dither clipped 168 samples; decrease volume?
sox WARN rate: rate clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 2 samples; decrease volume?
sox WARN rate: rate clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 2 samples; decrease volume?
sox WARN rate: rate clipped 2 samples; decrease volume?
sox WARN rate: rate clipped 4 samples; decrease volume?
sox WARN dither: dither clipped 3 samples; decrease volume?


  0%|          | 0/23 [00:00<?, ?it/s]

sox WARN rate: rate clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 2 samples; decrease volume?
sox WARN rate: rate clipped 40 samples; decrease volume?
sox WARN dither: dither clipped 37 samples; decrease volume?
sox WARN rate: rate clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 2 samples; decrease volume?
sox WARN rate: rate clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 2 samples; decrease volume?
sox WARN rate: rate clipped 2450 samples; decrease volume?
sox WARN dither: dither clipped 2166 samples; decrease volume?
sox WARN rate: rate clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 2 samples; decrease volume?
sox WARN rate: rate clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 2 samples; decrease volume?


[('pd_11', 'MZb0aofyoQ0'), ('pd_11', 'sCT__TK_AIg'), ('pd_11', 'mQLOO_Hk_nk'), ('pd_11', 'L-5-b3D4qG0'), ('pd_11', 'zNDqnzHUULE'), ('pd_11', '00X1n9TywVs'), ('pd_29', 'poeXwi54JzU'), ('pd_29', '3KTkfqk-OGw'), ('pd_29', 'XepnE-ize2Y'), ('pd_29', 'p4Cx5n7EXgA'), ('pd_29', 'zMmrNcdmdVY'), ('pd_29', 'YsmgPp_nlok'), ('pd_29', 'Dv8KvWZODKY'), ('pd_16', 'lJHSKcEXJGY'), ('pd_18', 's1N5MB8aD84'), ('pd_18', '6wcaVB8yNLM'), ('pd_18', 'VlCXrbP68jQ'), ('pd_18', 'TOh_QGc3TKA'), ('pd_18', 'zUa6-6RWgLo'), ('pd_18', 'en8o9Adb9dc'), ('pd_18', 'PkIza1FiYQQ'), ('pd_27', 'ukgRcNiKkUM'), ('pd_27', 'QZpknXvuv5k'), ('pd_27', 'u-Tbne3JFrg'), ('pd_27', '6-6l1-ldr3k'), ('pd_19', 'NwB2hKmHsE0'), ('pd_19', 'UYDflekDnYU'), ('pd_19', 'vxEPZiBqo68'), ('pd_19', 'uMFqq8J73EA'), ('pd_19', 'sSr6hQ4fxOw'), ('pd_19', 'chkoVJeQFpY'), ('pd_19', 'PHxx-e62zVI'), ('pd_19', 'vgjA3dEbzxI'), ('pd_26', '3TzTgCRiEr0'), ('pd_26', 'fy1Oi86wZL8'), ('pd_26', 'QdJEtqVQWBk'), ('pd_26', 'sYhpPtrqTiM'), ('pd_26', 'Nes-KRxJb3I'), ('pd_26', '

sox WARN rate: rate clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 2 samples; decrease volume?
